# <center><font color='darkblue'>Previsão de Rendimento de Colheita e Otimização da Irrigação</font></center>

### <font color='darkblue'>Projeto de Deep Learning com TensorFlow - Deploy via API</font>
### <font color='royalblue'>Beatriz Andrade</font>

## 🎯 O que este projeto faz?

Este modelo de Deep Learning prevê o **nível ideal de umidade do solo** para otimizar sistemas de irrigação agrícola. O objetivo é ajudar produtores a:

✅ Economizar água e energia  
✅ Maximizar a produtividade das colheitas  
✅ Tomar decisões baseadas em dados, não em "achismos"

---

## 🧠 Como funciona?

O modelo analisa 10 variáveis ambientais e agronômicas (radiação solar, precipitação, CO₂, profundidade das raízes, etc.) e aprende padrões complexos para prever a umidade ideal.

**Tecnologias:** TensorFlow/Keras (rede neural), StandardScaler (padronização), Early Stopping (otimização de treino)


## 📦 Instalando e Carregando os Pacotes

Vamos usar principalmente:
- **TensorFlow/Keras**: Para construir e treinar a rede neural
- **scikit-learn**: Para padronização dos dados e divisão treino/teste
- **pandas**: Para manipulação do dataset
- **joblib**: Para salvar o padronizador (necessário no deploy)

In [1]:
#!pip install -q -U watermark

In [2]:
#!pip install tensorflow

https://www.tensorflow.org/

In [3]:
# Imports
import joblib           # Salvar o padronizador em disco (necessário para API)
import sklearn
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split  # Dividir dados em treino/teste
from sklearn.preprocessing import StandardScaler      # Padronizar features (mesma escala)

# APIs do Keras (interface simplificada do TensorFlow)
from tensorflow.keras.models import Sequential        # Modelo sequencial (camadas empilhadas)
from tensorflow.keras.layers import Dense, Dropout    # Camadas densas e dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint  # Controle de treino

import warnings
warnings.filterwarnings('ignore')  # Esconder avisos desnecessários

In [4]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.20.0


In [5]:
# Só receber msgs de erro
%env TF_CPP_MIN_LOG_LEVEL=3

env: TF_CPP_MIN_LOG_LEVEL=3


In [6]:
%reload_ext watermark
%watermark -a "Beatriz Andrade"

Author: Beatriz Andrade



## 📂 Carregando o Conjunto de Dados

**Dataset:** 124 registros mensais de dados agrícolas simulados (10 anos)

**O que contém:**
- 10 variáveis preditoras (features): índice de vegetação, capacidade do solo, CO₂, nutrientes, etc.
- 1 variável alvo (target): **umidade ideal do solo** (%)

**Por que essas variáveis?** Cada uma representa um fator que afeta a necessidade de irrigação:
- Mais radiação solar → mais evaporação → mais água necessária
- Raízes profundas → acesso a água em camadas mais baixas
- Precipitação alta → menos irrigação artificial necessária

In [7]:
# Carregar o dataset
df_ba = pd.read_csv('dataset.csv')

## 🔍 Análise Exploratória

Antes de treinar o modelo, precisamos entender:
1. **Tipos de dados**: Tudo é numérico? Há textos/datas?
2. **Valores faltantes**: Existem células vazias?
3. **Distribuição**: Os valores estão em escalas muito diferentes?

**Por que isso importa?** Redes neurais são sensíveis a:
- Dados não numéricos (não conseguem processar texto diretamente)
- Valores faltantes (quebram o treinamento)
- Escalas muito diferentes (uma variável pode "dominar" o aprendizado)

In [8]:
df_ba.shape

(124, 12)

In [9]:
df_ba.head()

,data,indice_vegetacao,capacidade_solo,concentracao_co2,nivel_nutrientes,indice_fertilizantes,profundidade_raiz,radiacao_solar,precipitacao,estagio_crescimento,historico_rendimento,umidade
0,2012-12-01,323,455,3102.61,423.45,844.0,468.0,578.0,28.67,207.70504,117.7,79.261905
1,2013-01-01,345,546,3100.45,415.85,799.0,485.0,557.0,24.49,228.94287,4.5,82.193548
2,2013-02-01,362,595,3199.41,410.77,718.0,466.0,552.0,22.06,238.41747,25.1,74.839286
3,2013-03-01,376,636,3281.67,414.82,614.0,442.0,574.0,21.64,218.47599,53.6,77.935484
4,2013-04-01,383,738,3261.65,451.04,619.0,429.0,595.0,22.30,226.15010,166.0,80.450000


In [10]:
df_ba.tail()

,data,indice_vegetacao,capacidade_solo,concentracao_co2,nivel_nutrientes,indice_fertilizantes,profundidade_raiz,radiacao_solar,precipitacao,estagio_crescimento,historico_rendimento,umidade
119,2022-11-01,362,363,2626.91,1252.78,738.07,427.49,1430.48,60.18,186.68326,38.2,77.950000
120,2022-12-01,310,322,2736.64,1287.68,749.57,385.09,1472.27,62.25,210.72987,33.7,76.177419
121,2023-01-01,277,307,2842.81,1289.12,761.60,373.03,1525.43,63.04,244.41912,4.6,74.774194
122,2023-02-01,323,330,2936.19,1303.59,759.59,390.69,1572.25,71.52,223.31732,6.9,66.910714
123,2023-03-01,360,339,2847.84,1234.88,771.62,396.87,1302.61,74.80,228.56676,41.5,69.000000


In [11]:
# Verificar os tipos de dados das colunas
df_ba.dtypes

# O que estamos procurando:
# - int64/float64: OK para o modelo ✅
# - object: Precisa ser tratado (pode ser texto/data) ⚠️

data                     object
indice_vegetacao          int64
capacidade_solo           int64
concentracao_co2        float64
nivel_nutrientes        float64
indice_fertilizantes    float64
profundidade_raiz       float64
radiacao_solar          float64
precipitacao            float64
estagio_crescimento     float64
historico_rendimento    float64
umidade                 float64
dtype: object

In [12]:
# Identificar colunas não numéricas
non_numeric_columns = df_ba.select_dtypes(include=['object']).columns
print(f'Colunas não numéricas: {non_numeric_columns}')

# Resultado: apenas 'data' (que é como um "ID" do registro)
# ❌ Não devemos incluir IDs no modelo - eles não têm poder preditivo

Colunas não numéricas: Index(['data'], dtype='object')


## 🧹 Limpeza e Transformação

### Por que remover a coluna 'data'?

Ela funciona como um **identificador único** (ID), não como uma variável preditiva.

**Decisão:** Se quiséssemos capturar **sazonalidade** (variações por mês/ano), extrairíamos features como `mes`, `trimestre`, `ano`. Mas nosso dataset é pequeno (124 pontos) e já tem variáveis temporais indiretas (estágio de crescimento).

---

### Verificação de valores faltantes

Mesmo que o dataset pareça completo, sempre validamos — valores `NaN` (Not a Number) quebram o treinamento da rede neural.

In [13]:
# Remover coluna não numérica (funciona como ID, não como feature preditiva)
df_ba = df_ba.drop(columns=non_numeric_columns)

# Agora temos apenas colunas numéricas que o modelo pode processar

In [14]:
# Verificar se a coluna 'umidade' contém valores numéricos
if df_ba['umidade'].dtype == 'object':
    df_ba['umidade'] = pd.to_numeric(df_ba['umidade'], errors='coerce')
    
# errors='coerce': Se encontrar texto, converte para NaN (em vez de quebrar)
# No nosso caso, 'umidade' já é float64, então isso é só garantia

In [15]:
# Remover linhas com valores faltantes
df_ba = df_ba.dropna()

# Por que dropna() e não preencher (imputation)?
# - Dataset pequeno (124 registros): não podemos "inventar" dados
# - Se houvesse muitos NaN, consideraríamos preencher com média/mediana
# - No nosso caso, não há NaN, então essa linha é só precaução

## Preparação

In [16]:
# Separar preditores (X) e variável alvo (y)
X = df_ba.drop(columns='umidade')  # Todas as colunas MENOS umidade
y = df_ba['umidade']                # Só a umidade (o que queremos prever)

# X: (124, 10) - 124 registros, 10 features
# y: (124,)    - 124 valores de umidade

## ⚖️ Padronização dos Dados

### O que é padronização (StandardScaler)?

Transforma cada variável para ter:
- **Média = 0**
- **Desvio padrão = 1**

**Fórmula:** `(valor - média) / desvio_padrão`

---

### Por que isso é CRÍTICO em redes neurais?

Imagine que temos:
- `radiacao_solar`: valores entre 500-1500 W/m²
- `precipitacao`: valores entre 20-75 mm

Sem padronização, a rede neural "pensa" que radiação é 10-20x mais importante só por ter números maiores.

---

### Por que salvar o scaler (joblib.dump)?

Quando formos fazer previsões novas (deploy via API), precisamos **aplicar EXATAMENTE a mesma transformação** que usamos no treino.

Se treinarmos com dados padronizados mas previrmos com dados brutos, o modelo vai errar completamente.

In [17]:
# Dividir os dados em conjunto de treino (80%) e teste (20%)
X_treino, X_teste, y_treino, y_teste = train_test_split(
    X, y, 
    test_size=0.2,      # 20% para teste (25 registros)
    random_state=42     # Semente para reprodutibilidade
)

# Por que 80/20?
# - 80% (99 registros): Suficiente para o modelo aprender padrões
# - 20% (25 registros): Suficiente para validar performance real
# - Com datasets maiores, poderíamos usar 90/10

In [18]:
# Cria o padronizador
scaler = StandardScaler()

# Padroniza os dados
X_treino_scaled = scaler.fit_transform(X_treino)  # Aprende E transforma treino
X_teste_scaled = scaler.transform(X_teste)        # Só transforma teste (usa parâmetros do treino)

# ⚠️ IMPORTANTE: 
# - fit_transform() no TREINO: calcula média/desvio E aplica
# - transform() no TESTE: usa média/desvio do TREINO (evita vazamento de dados)

In [19]:
# Salva o padronizador no disco (necessário para o deploy via API)
joblib.dump(scaler, 'scaler_ba.joblib')

# Quando formos fazer previsões novas:
# scaler = joblib.load('scaler_ba.joblib')
# novos_dados_scaled = scaler.transform(novos_dados)

['scaler_ba.joblib']

## 🏗️ Construção do Modelo

### Arquitetura da Rede Neural

Vamos construir uma rede neural **feedforward** (dados fluem em uma direção: entrada → camadas ocultas → saída).

**Estrutura escolhida:**

---

### Por que essa arquitetura?

#### 🔹 **Camadas Decrescentes (64 → 32 → 16)**
**O que faz:** Cada camada tem menos neurônios que a anterior  
**Por que:** Funciona como um funil — as primeiras camadas capturam padrões gerais, as últimas refinam para a previsão final

**Analogia para entrevista:** "É como resumir um texto: você começa com todas as informações (64), vai destacando o importante (32), e termina com a essência (16 → 1 valor final)."

---

#### 🔹 **Função de Ativação ReLU**
**O que faz:** `ReLU(x) = max(0, x)` — se valor negativo, zera; se positivo, mantém  
**Por que:** Permite que a rede aprenda relações **não-lineares** (complexas)

**Sem ReLU:** O modelo seria uma regressão linear glorificada  
**Com ReLU:** Pode aprender que "umidade ideal depende de radiação solar ATÉ certo ponto, depois estabiliza"


---

#### 🔹 **Dropout (30%)**
**O que faz:** Desliga aleatoriamente 30% dos neurônios a cada iteração de treino  
**Por que:** Força a rede a **não decorar** os dados — obriga todos os neurônios a aprenderem padrões reais

**Sem Dropout:** O modelo pode ter 99% de acurácia no treino e 60% no teste (overfitting)  
**Com Dropout:** Performance mais equilibrada entre treino e teste

**Por que 30%?** É um valor padrão que funciona bem — menos que 20% tem pouco efeito, mais que 50% pode atrapalhar o aprendizado.

---

#### 🔹 **Camada Final: Dense(1)**
**Por que 1 neurônio?** Queremos prever **1 valor contínuo** (umidade em %)  
**Sem ativação:** Queremos o valor real (ex: 73.5%), não uma probabilidade entre 0-1

Se fosse classificação (ex: "umidade baixa/média/alta"), usaríamos 3 neurônios + softmax.

In [20]:
# Definir a arquitetura do modelo
modelo_ba = Sequential([
    # Camada 1: 64 neurônios
    Dense(64, activation='relu', input_shape=(X_treino.shape[1],)),
    # input_shape: (10,) porque temos 10 features
    # activation='relu': permite aprender padrões não-lineares
    
    Dropout(0.3),  # Desliga 30% dos neurônios (evita overfitting)
    
    # Camada 2: 32 neurônios (funil - reduzindo dimensionalidade)
    Dense(32, activation='relu'),
    Dropout(0.3),
    
    # Camada 3: 16 neurônios (refinamento final)
    Dense(16, activation='relu'),
    
    # Camada de saída: 1 neurônio (valor contínuo de umidade)
    Dense(1)  # Sem ativação = regressão (predizer valor real)
])

# Total de parâmetros treináveis: 3.329
# São os "pesos" que o modelo vai ajustar durante o treinamento

## ⚙️ Compilação do Modelo

Compilar = configurar **como** o modelo vai aprender.

Precisamos definir 3 coisas:
1. **Optimizer**: Como ajustar os pesos (algoritmo de aprendizado)
2. **Loss**: Como medir o erro (função de perda)
3. **Metrics**: O que monitorar durante o treino (além da loss)

---

### 🔹 **Optimizer: Adam**

**O que faz:** Ajusta os pesos da rede para minimizar o erro (loss)

**Por que Adam?**
- Auto-ajusta a taxa de aprendizado (learning rate adaptativo)
- Combina vantagens do SGD com Momentum + RMSProp
- Funciona bem "out of the box" na maioria dos casos

**Alternativas:**
- SGD: Mais simples, mas requer tuning manual do learning rate
- RMSProp: Bom, mas Adam geralmente é superior

---

### 🔹 **Loss: MSE (Mean Squared Error)**

**O que faz:** Calcula `(previsão - valor_real)²`, depois tira a média

**Por que MSE?**
- **Penaliza erros grandes** mais fortemente (pelo quadrado)
- Errar 20 pontos é 4x pior que errar 10 pontos (10² = 100 vs 20² = 400)
- Isso é bom para irrigação: prever 50% quando o ideal é 90% é crítico

**Alternativa:** MAE (Mean Absolute Error) — trata todos os erros igualmente

**Decisão:** MSE como loss porque queremos punir erros grandes; MAE como métrica porque é mais fácil de interpretar.

---

### 🔹 **Metric: MAE (Mean Absolute Error)**

**O que faz:** Calcula `|previsão - valor_real|`, depois tira a média

**Por que MAE?**
- **Mais intuitiva**: "O modelo erra, em média, X pontos de umidade"
- MSE é ótima para treinar, mas difícil de interpretar (está em "pontos ao quadrado")

**Exemplo:**
- MAE = 12.5 → "O modelo erra, em média, 12.5 pontos de umidade"
- MSE = 232 → "O que isso significa?" (não é óbvio)

In [21]:
# Compilar o modelo
modelo_ba.compile(
    optimizer='adam',  # Algoritmo de otimização (ajusta pesos automaticamente)
    loss='mse',        # Função de perda (MSE penaliza erros grandes)
    metrics=['mae']    # Métrica para monitorar (MAE é mais interpretável)
)

# Agora o modelo está pronto para treinar!
# Próximo passo: definir callbacks (Early Stopping, Model Checkpoint)

## 🎮 Definindo os Callbacks

**Callbacks** = Funções que executam durante o treinamento para **controlar o processo**.

Vamos usar 2 callbacks essenciais:

---

### 🔹 **EarlyStopping**

**O que faz:** Para o treinamento se o modelo não melhorar por X épocas

**Parâmetros:**
- `monitor='val_loss'`: Monitora a perda no conjunto de **validação**
- `patience=10`: Espera 10 épocas sem melhoria antes de parar
- `restore_best_weights=True`: Restaura os pesos da melhor época (não os últimos)

**Por que isso é importante?**
- **Economiza tempo**: Se o modelo já aprendeu tudo, não adianta treinar 100 épocas
- **Evita overfitting**: Depois de certo ponto, o modelo começa a "decorar" os dados de treino

**Por que patience=10?**
- Dataset pequeno (99 registros de treino): o modelo pode ter variações temporárias
- Patience muito baixa (ex: 3): pode parar prematuramente
- Patience muito alta (ex: 30): perde o benefício do early stopping

---

### 🔹 **ModelCheckpoint**

**O que faz:** Salva automaticamente o **melhor modelo** durante o treino

**Parâmetros:**
- `'modelo_ba.keras'`: Nome do arquivo onde salvar
- `save_best_only=True`: Só salva quando o modelo melhora (economiza espaço)

**Por que isso é importante?**
- O treinamento acontece na memória RAM (se der erro, você perde tudo)
- Se o treino trava na época 87, você não perde o trabalho — tem o checkpoint da melhor época

In [22]:
# Callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',           # Monitora a perda de validação
    patience=10,                  # Espera 10 épocas sem melhoria
    restore_best_weights=True     # Restaura os pesos da melhor época
)

model_checkpoint = ModelCheckpoint(
    'modelo_ba.keras',    # Nome do arquivo para salvar
    save_best_only=True   # Só salva quando o modelo melhora
)

# Esses callbacks serão passados para o método .fit()

## 🏋️ Treinamento do Modelo

Agora vamos treinar o modelo com os dados de treino.

**O que acontece no `.fit()`:**

1. **Forward Pass**: Dados passam pelas camadas, modelo faz previsões
2. **Cálculo do Erro**: Compara previsões com valores reais (MSE)
3. **Backward Pass (Backpropagation)**: Calcula como cada peso contribuiu para o erro
4. **Atualização dos Pesos**: Optimizer (Adam) ajusta os pesos para reduzir o erro
5. **Repete** para todos os batches, em todas as épocas

---

### 🔹 **Parâmetros do Treinamento**

#### **validation_split=0.2**
**O que faz:** Separa 20% dos dados de **treino** para validação  
**Por que:** Permite monitorar overfitting durante o treino (sem tocar nos dados de teste)

**Divisão final:**
- Treino: 80% de 99 = 79 registros
- Validação: 20% de 99 = 20 registros
- Teste: 25 registros (separados lá em cima)

---

#### **epochs=100**
**O que é:** 1 época = o modelo vê todos os dados de treino uma vez  
**Por que 100:** Limite máximo — mas o Early Stopping pode parar antes

---

#### **batch_size=32**
**O que é:** Quantos exemplos o modelo vê antes de atualizar os pesos

**Por que 32?**
- Batch muito pequeno (1): Atualiza pesos a cada exemplo → treino instável, mas rápido
- Batch muito grande (79): Atualiza pesos 1x por época → treino lento, mas estável
- 32 é o "meio termo" padrão da indústria


**Cálculo:** Com 79 registros de treino e batch_size=32:
- 79 ÷ 32 = 2.46 → 3 batches por época
- Última iteração tem apenas 15 exemplos (79 - 32 - 32 = 15)

In [23]:
# Treinar o modelo
history = modelo_ba.fit(
    X_treino_scaled,           # Dados de entrada (padronizados)
    y_treino,                   # Valores alvo (umidade)
    validation_split=0.2,       # 20% dos dados de treino para validação
    epochs=100,                 # Máximo de 100 épocas (Early Stopping pode parar antes)
    batch_size=32,              # 32 exemplos por batch
    callbacks=[early_stopping, model_checkpoint]  # Early Stopping + Checkpoint
)

# O que acontece:
# - Época 1-48: Modelo aprende padrões, loss diminui
# - Época 49: Melhor performance (val_loss = 236.42)
# - Época 49-59: Modelo tenta melhorar, mas não consegue (patience=10)
# - Época 59: Early Stopping ativa, treino para
# - Pesos restaurados para época 49 (melhor performance)

Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 95ms/step - loss: 5204.5420 - mae: 72.0015 - val_loss: 5182.3101 - val_mae: 71.8880
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 5177.7739 - mae: 71.8125 - val_loss: 5159.4858 - val_mae: 71.7295
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 5155.2251 - mae: 71.6562 - val_loss: 5135.6572 - val_mae: 71.5635
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 5117.3677 - mae: 71.3876 - val_loss: 5110.0000 - val_mae: 71.3844
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 5106.1665 - mae: 71.3128 - val_loss: 5082.6006 - val_mae: 71.1925
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 5070.8628 - mae: 71.0658 - val_loss: 5053.1587 - val_mae: 70.9855
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 5022.6099 - mae: 70.7201 - val_loss: 5020.9541 - val_mae: 70.7583
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 4997.2764 - mae: 70.5339 - val_loss: 4985.4072 - val_mae: 70.5066
Epoch 9/

### 📊 Interpretando o Output do Treinamento

Vamos analisar o que aconteceu:

**Época 1:**
```
loss: 5204.5420 - mae: 72.0015 - val_loss: 5182.3101 - val_mae: 71.8880
```
- Modelo começou "perdido" (erro médio de 72 pontos de umidade)
- Perda alta porque os pesos são inicializados aleatoriamente

**Época 44:**
```
loss: 433.6098 - mae: 16.6587 - val_loss: 193.1488 - val_mae: 11.8212
```
- ✅ **Melhor desempenho**: val_loss = 193.14
- MAE de validação = 11.82 (erro médio de ~12 pontos)

**Épocas 45-54:**
```
val_loss: 199.62, 211.66, 244.90...(subindo)
```
- Modelo não consegue mais melhorar
- val_loss começa a **aumentar** (sinal de overfitting)
- Early Stopping conta: 1, 2, 3... 10 → **PARA**

---

### 🎯 **Resultado Final**

✅ **Treino parou na época 54* (de 100 possíveis)  
✅ **Pesos restaurados da época 44** (melhor val_loss)  
✅ **Economizou 56 épocas** de treino desnecessário  
✅ **Evitou overfitting** (modelo não começou a decorar os dados)

**Conclusão:** O Early Stopping funcionou perfeitamente!

## 📊 Avaliação do Modelo

Agora vamos testar o modelo nos **dados de teste** (25 registros que ele NUNCA viu).

**Por que avaliar no teste?**
- Dados de **treino**: O modelo viu durante o aprendizado (pode ter decorado)
- Dados de **validação**: O modelo viu indiretamente (Early Stopping usou)
- Dados de **teste**: O modelo NUNCA viu — é a métrica mais honesta

In [24]:
# Avaliar o modelo no conjunto de teste
teste_loss, teste_mae = modelo_ba.evaluate(X_teste_scaled, y_teste)

# evaluate() retorna:
# - teste_loss: MSE nos dados de teste
# - teste_mae: MAE nos dados de teste (mais interpretável)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 192.4586 - mae: 12.0120


In [25]:
print(f'Teste Loss (MSE): {teste_loss:.2f}')
print(f'Teste MAE: {teste_mae:.2f}')

Teste Loss (MSE): 192.46
Teste MAE: 12.01


### 🎯 Interpretando as Métricas

**MAE = 12.01**
- O modelo erra, em média, **12.01 pontos de umidade**
- Se o ideal é 75%, o modelo pode prever entre 63% e 87%

**Isso é bom ou ruim?**
- ✅ Para um sistema de irrigação automatizado: **aceitável**
- ✅ Melhor que humanos "chutando" baseado em experiência
- ⚠️ Mas tem espaço para melhorias (mais dados, mais features)

---

**MSE = 192.46**
- Métrica quadrática (penaliza erros grandes)
- Mais difícil de interpretar que MAE
- Usada durante o treino, mas para reporting usamos MAE

---

### 📈 Comparação Treino vs Validação vs Teste

| Conjunto | MAE | Interpretação |
|----------|-----|---------------|
| Validação (época 44) | 11.82 | Performance durante treino |
| Teste (avaliação final) | 12.01 | Performance em dados nunca vistos |

✅ **Resultados muito próximos** → Modelo generalizou bem (não fez overfitting)

Se fosse:
- MAE validação = 12.01, MAE teste = 25.00 → ❌ Overfitting (decorou os dados)
- MAE validação = 50.00, MAE teste = 48.00 → ❌ Underfitting (não aprendeu)

Vamos agora fazer o deploy do modelo via API.

In [26]:
%reload_ext watermark
%watermark -a "Beatriz Andrade"

Author: Beatriz Andrade



In [27]:
%watermark -v -m

Python implementation: CPython
Python version       : 3.13.5
IPython version      : 8.30.0

Compiler    : MSC v.1929 64 bit (AMD64)
OS          : Windows
Release     : 11
Machine     : AMD64
Processor   : AMD64 Family 25 Model 80 Stepping 0, AuthenticAMD
CPU cores   : 16
Architecture: 64bit



In [28]:
%watermark --iversions

sklearn   : 1.7.1
joblib    : 1.4.2
keras     : 3.12.0
tensorflow: 2.20.0
pandas    : 2.3.0



# Fim